In [1]:
import os
import numpy as np

import torch
torch.manual_seed(29)
from torch import nn
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
import torch.nn.parallel
cudnn.benchmark = True
from torch.utils.data import DataLoader

from torchstat import stat
from utils import  models
from utils.data import dataset_1
from utils.trainer_utils import parfilter

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from skimage.measure import compare_psnr, compare_ssim

from matplotlib import pyplot as plt

from glob import glob
from tqdm import tqdm

from torch.nn import functional as F
from PIL.PngImagePlugin import PngImageFile, PngInfo

In [2]:
def image_quality_comp(I, Iout, Enout):
    
    for x in ['test.png', 'test_35.jpg', 'test.flif', 'test1.png', 'test1.flif', 'test_comp.png']: 
        if os.path.exists(x):os.remove(x)
    
    I.save('test.png')
    Iout.save('test_comp.png')
    I.save('test_10.jpg', quality = 10)
    Enout.save('test1.png')
    
    os.system("flif -e test.png test.flif")
    os.system("flif -e test1.png test1.flif")
    
    print('Original Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test.png')/(I.size[0] * I.size[1])))
    print('JPG-10% :: ' + str(8*os.path.getsize('test_10.jpg')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test.flif')/(I.size[0] * I.size[1])))
    print('Encoded Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test1.png')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test1.flif')/(I.size[0] * I.size[1])))
    
def image_quality_comp2(I, Iout):
    
    for x in ['test.png', 'test_35.jpg', 'test.flif', 'test1.png', 'test1.flif', 'test_comp.png']: 
        if os.path.exists(x):os.remove(x)
    
    I.save('test.png')
    Iout.save('test_comp.png')
    I.save('test_10.jpg', quality = 10)
    
    os.system("flif -e test.png test.flif")
    
    print('Original Image :: ')
    print('PNG     :: ' + str(8*os.path.getsize('test.png')/(I.size[0] * I.size[1])))
    print('JPG-10% :: ' + str(8*os.path.getsize('test_10.jpg')/(I.size[0] * I.size[1])))
    print('FLIF    :: ' + str(8*os.path.getsize('test.flif')/(I.size[0] * I.size[1])))

In [3]:
def compress_and_decompress(img, model, src_fldr, dst_fldr):
    I_org = Image.open(img).convert('RGB')
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(256*np.ceil(I.shape[0]/256) - I.shape[0])), 
                   (0, int(256*np.ceil(I.shape[1]/256) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    s_ = 256
    d_ = 64
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            X = model(It.cuda())
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    
    Iout.save(img.replace(src_fldr, dst_fldr))
    return Iout

In [4]:
def perform_compression(I_org, model):
    
    w, h = I_org.size
    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(256*np.ceil(I.shape[0]/256) - I.shape[0])), 
                   (0, int(256*np.ceil(I.shape[1]/256) - I.shape[1])), 
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)
    
    
    s_ = 512
    d_ = 512//4
    
    c_ = s_//d_
    
    I1 = np.float32(I)#[:256, :256, :]
    I1 = np.transpose(I1, [2, 0, 1])
    Iout = np.zeros_like(I1)
    Enout = np.zeros((3, I1.shape[1]//c_, I1.shape[2]//c_))
    for i in range(0, I1.shape[1], s_):
        for j in range(0, I1.shape[2], s_):
            It = torch.from_numpy(np.expand_dims(I1[:, i:i+s_, j:j+s_], 0))/255.0
            Xe = model.encode(It.cuda())
            X = model.decode(Xe)
            X = X.data.squeeze().cpu().numpy()
            Iout[:, i:i+s_, j:j+s_] = np.clip(X, 0, 1)
            Xe = (Xe + 1)/2
            Enout[:, i//c_:(i+s_)//c_, j//c_:(j+s_)//c_] = Xe.data.squeeze().cpu().numpy()
    
    
    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Enout = np.uint8(255 * Enout.transpose([1, 2, 0]))
    
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))
    Enout = Image.fromarray(Enout)
    
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Iout).copy())
#     ssim = compare_ssim(np.uint8(I_org).copy(), np.uint8(Iout).copy())
    
    print('PSNR-PROP :: ' + "{0:0.02f}".format(psnr))
#     print('SSIM-PROP :: ' + "{0:0.02f}".format(ssim), multichannel=True)
    
    image_quality_comp(I_org, Iout, Enout)
    psnr = compare_psnr(np.uint8(I_org).copy(), np.uint8(Image.open('test_10.jpg')).copy())
    print('PSNR-JPG10 :: ' + "{0:0.02f}".format(psnr))
    
    
    return Iout, Enout

In [112]:
def compress(I_org, model):

    e_ = 512
    c_ = 4
    d_ = e_ // c_
    pad_ = 4

    w, h = I_org.size

    comp_w_new = np.ceil(w/c_)
    comp_h_new = np.ceil(h/c_)

    new_w = int(e_ * np.ceil(w/e_))
    new_h = int(e_ * np.ceil(h/e_))

    com_w = new_w // c_
    com_h = new_h // c_

    I = np.uint8(I_org).copy()
    I = np.pad(I, ((0, int(new_h - h)),
                   (0, int(new_w - w)),
                   (0, 0)), mode = "reflect")
    I = Image.fromarray(I)


    I1 = np.float32(I)/255.0
    I1 = np.transpose(I1, [2, 0, 1])

    Enout = np.zeros((3, com_h, com_w))
    Enout_w = np.zeros((3, com_h, com_w))
    for i in list(np.arange(0, new_h, e_)):
        for j in list(np.arange(0, new_w, e_)):
            if i == 0:
                x1 = int(i)
                x2 = int((i + e_) + (pad_*2*c_))
            else:
                x1 = int(i - (pad_*c_))
                x2 = int((i + e_) + (pad_*c_))

            if j == 0:
                y1 = int(j)
                y2 = int((j + e_) + (pad_*2*c_))
            else:
                y1 = int(j - (pad_*c_))
                y2 = int((j + e_) + (pad_*c_))
            It = torch.from_numpy(np.expand_dims(I1[:, x1:x2, y1:y2], 0))
            Xe = model.encode(It.cuda())
            Xe = (Xe + 1.0)/2.0
            Enout[:, x1//c_:x2//c_, y1//c_:y2//c_] += Xe.data.squeeze().cpu().numpy()
            Enout_w[:, x1//c_:x2//c_, y1//c_:y2//c_] += 1.0

    Enout = Enout/Enout_w
    Enout = np.uint8(255 * Enout.transpose([1, 2, 0]))

    Enout = Image.fromarray(Enout).crop((0, 0, comp_w_new, comp_h_new))

    return Enout


def decompress(EnIn, model, w, h):

    e_ = 512
    c_ = 4
    d_ = e_ // c_
    pad_ = 2

#     w, h = int(EnIn.text['w']), int(EnIn.text['h'])

    comp_w_new = np.ceil(w/c_)
    comp_h_new = np.ceil(h/c_)

    new_w = int(e_ * np.ceil(w/e_))
    new_h = int(e_ * np.ceil(h/e_))

    com_w = new_w // c_
    com_h = new_h // c_


    Iout = np.zeros((3,new_h,new_w), dtype = np.float32)
    Iout_w = np.zeros((3,new_h,new_w), dtype = np.float32)

    EnIn = np.uint8(EnIn).copy()
    EnIn = np.pad(EnIn, ((0, int(com_h - EnIn.shape[0])),
                         (0, int(com_w - EnIn.shape[1])),
                         (0, 0)), mode = "reflect")


    EnIn = np.float32(EnIn)/255.0
    EnIn = np.transpose(EnIn, [2, 0, 1])
    for i in list(np.arange(0, com_h, d_)):
        for j in list(np.arange(0, com_w, d_)):

            if i == 0:
                x1 = int(i)
                x2 = int((i + d_) + pad_*2)
            else:
                x1 = int(i - pad_)
                x2 = int((i + d_) + pad_)

            if j == 0:
                y1 = int(j)
                y2 = int((j + d_) + pad_*2)
            else:
                y1 = int(j - pad_)
                y2 = int((j + d_) + pad_)

            It = torch.from_numpy(np.expand_dims(EnIn[:, x1:x2, y1:y2], 0))
            It = It * 2.0 - 1.0
            Xe = model.decode(It.cuda()).data.squeeze().cpu()

            Iout[:, x1*c_:x2*c_, y1*c_:y2*c_] += np.clip(Xe.numpy(), 0, 1)
            Iout_w[:, x1*c_:x2*c_, y1*c_:y2*c_] += 1.0

    Iout = Iout/Iout_w

    Iout = np.uint8(255 * Iout.transpose([1, 2, 0]))
    Iout = Image.fromarray(Iout).crop((0, 0, w, h))

    return Iout


In [71]:
img_file = "/home/narsi/Downloads/openjpeg-v2.3.1-linux-x86_64/bin/alberto-montalesi-176097.tif"

In [72]:
I = Image.open(img_file).convert("RGB")

In [8]:
model = models.QuantACTShuffleV6()
check_point_file = "/media/narsi/LargeData/SP_2020/compressACT/weights/"+\
"QuantACTShuffleV6_exp01/checkpoint.pth.tar"
checkpoint = torch.load(check_point_file)
model.load_state_dict(checkpoint['state_dict'], strict = True)
model.cuda()
model.eval()
print('.')

.


In [16]:
Iout, Enout = perform_compression(I, model)

PSNR-PROP :: 35.23
Original Image :: 
PNG     :: 10.80944081959707
JPG-10% :: 0.19121737637362637
FLIF    :: 7.085416666666666
Encoded Image :: 
PNG     :: 0.9992931547619047
FLIF    :: 0.9450606684981685
PSNR-JPG10 :: 30.31


In [5]:
model = models.QuantACTShuffleV7()
model.cuda()
%timeit model.encode(torch.randn(1, 3, 256, 256).cuda())
%timeit model.decode(torch.randn(1, 3, 32, 32).cuda())

2.09 ms ± 10.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.54 ms ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
model = models.QuantACTShuffleV7()
%timeit model.encode(torch.randn(1, 3, 256, 256))
%timeit model.decode(torch.randn(1, 3, 32, 32))

17.3 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
31.3 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
src_fldr = "/media/narsi/LargeData/DATASETS/superrez/clic_2019/professional_valid/valid"
dst_fldr = "/media/narsi/LargeData/DATASETS/superrez/clic_2019/professional_valid/model_test"

imgs = glob(src_fldr + os.sep + "*.png")

In [8]:
model.cuda()
print('.')

.


In [100]:
for img in tqdm(imgs):
    compress_and_decompress(img, model, src_fldr, dst_fldr)

NameError: name 'imgs' is not defined

In [11]:
compressed_imgs = [img.replace(src_fldr, dst_fldr) for img in imgs]

In [12]:
from utils.competition_eval import evaluate2

In [13]:
results = evaluate2(compressed_imgs, imgs)

41it [01:36,  2.36s/it]


In [14]:
results

{'PSNR': 29.980508816961674, 'MSSSIM': 0.968473254012289}

In [73]:
jpeg2000_comp = "/home/narsi/Downloads/openjpeg-v2.3.1-linux-x86_64/bin/opj_compress"
jpeg2000_decomp = "/home/narsi/Downloads/openjpeg-v2.3.1-linux-x86_64/bin/opj_decompress"
src_enco_img = "/home/narsi/local/1WeStar/test_en.png"
src_jp2_img = "/home/narsi/local/1WeStar/test_en_comp.jp2"
src_dejp2_img = "/home/narsi/local/1WeStar/test_en_decomp.png"

In [75]:
Enout = compress(I, model)
metadata = PngInfo()
metadata.add_text("w", str(I.size[0]))
metadata.add_text("h", str(I.size[1]))
Enout.save("test_en.png", pnginfo=metadata)
Enout = Image.open("test_en.png")

# Iout = decompress(Enout, model)

In [113]:
compress_ = []
for i in [11]:
    os.system(jpeg2000_comp + " -i " + src_enco_img + " -o " + src_jp2_img+ " -r " + str(i) + " -b  64,64")#-i 
    #-q " + str(i) + " -m 6
    os.system(jpeg2000_decomp + " -i " + src_jp2_img + " -o " + src_dejp2_img)#-i 
    Enout_ = Image.open(src_dejp2_img)
    Iout = decompress(Enout_, model, I.size[0], I.size[1])
    p_ = compare_psnr(np.uint8(I).copy(), np.uint8(Iout).copy())
    bpp_ = str(8*os.path.getsize(src_jp2_img)/(I.size[0] * I.size[1]))
    compress_.append([i, p_, bpp_])
    print([i, p_, bpp_])
    Iout.save("test_q_" + str(i) + ".png")

[11, 34.23204259511609, '0.1363777054292003']


In [41]:
import sewar

In [109]:
for i in [11]:
    print(sewar.msssim(np.uint8(I).copy(), np.uint8(Image.open("test_q_" + str(i) + ".png")).copy()))

(0.9540365745746157+0j)


In [45]:
[5, 34.55272286482678, '0.300380608974359']
[10, 33.62807094428767, '0.1501058836996337']
[15, 32.731314515452034, '0.10012305402930403']
[20, 31.954220587188562, '0.0751860119047619']
[25, 31.386938633765247, '0.06011332417582418']
[30, 30.852527248242634, '0.05007154304029304']
[35, 30.362834335900537, '0.04296302655677656']
[40, 29.937515171854972, '0.03733974358974359']
[45, 29.712449219812854, '0.033407738095238095']
[50, 29.483821768748406, '0.03005666208791209']
[55, 29.208002184814802, '0.027229281135531136']
[60, 29.024169793349355, '0.02497710622710623']
[65, 28.787706241216036, '0.02306261446886447']
[70, 28.695274113517897, '0.02141426282051282']
[75, 28.56900332863841, '0.020086423992673994']
[80, 28.40455015191608, '0.01883871336996337']
[85, 28.291059015526017, '0.0176510989010989']
[90, 28.196563595891835, '0.016738209706959706']
[95, 28.02455495634968, '0.015811011904761904']
[100, 27.87443596587932, '0.015032623626373627']

## PSNR

[50, 35.073917919712535, '0.5857772435897436']
[45, 34.89198652720193, '0.4241300366300366']
[40, 34.36491486994174, '0.23538804945054945']
[35, 32.53138783360557, '0.09493761446886446']
[30, 29.396947787729346, '0.02867731227106227']

[30, 29.396947787729346, '0.02867731227106227']